In [6]:
import arcpy
import pandas as pd

# Feature class de entrada
fc = r"C:\Users\diana\Documents\ArcGIS\Projects\INTEIA\INTEIA.gdb\CLC_2020_Intersect_tiles"

# Leer campos necesarios
fields = ['tile_id', 'nivel_3', 'AREA_HECTAREAS']
rows = [row for row in arcpy.da.SearchCursor(fc, fields)]
df = pd.DataFrame(rows, columns=fields)

# Calcular área total por tile para luego obtener proporción por clase
total_area = df.groupby('tile_id')['AREA_HECTAREAS'].sum().reset_index()
total_area = total_area.rename(columns={'AREA_HECTAREAS': 'AREA_TOTAL'})
df = df.merge(total_area, on='tile_id')
df['PCT_AREA'] = df['AREA_HECTAREAS'] / df['AREA_TOTAL']

# Diccionario de leyendas (como antes)
leyendas = {
    '111': 'Tejido urbano continuo',
    '112': 'Tejido urbano discontinuo',
    '121': 'Zonas industriales o comerciales',
    '122': 'Red vial, ferroviaria y terrenos asociados',
    '123': 'Zonas portuarias',
    '124': 'Aeropuertos',
    '125': 'Obras hidráulicas',
    '131': 'Zonas de extracción minera',
    '132': 'Zonas de disposición de residuos',
    '141': 'Zonas verdes urbanas',
    '142': 'Instalaciones recreativas',
    '211': 'Otros cultivos transitorios',
    '212': 'Cereales',
    '213': 'Oleaginosas y leguminosas',
    '214': 'Hortalizas',
    '215': 'Tubérculos',
    '221': 'Cultivos permanentes herbáceos',
    '222': 'Cultivos permanentes arbustivos',
    '223': 'Cultivos permanentes arbóreos',
    '224': 'Cultivos agroforestales',
    '225': 'Cultivos confinados',
    '231': 'Pastos limpios',
    '232': 'Pastos arbolados',
    '233': 'Pastos enmalezados',
    '241': 'Mosaico de cultivos',
    '242': 'Mosaico de pastos y cultivos',
    '243': 'Mosaico de cultivos, pastos y espacios naturales',
    '244': 'Mosaico de pastos con espacios naturales',
    '245': 'Mosaico de cultivos y espacios naturales',
    '311': 'Bosque denso',
    '312': 'Bosque abierto',
    '313': 'Bosque fragmentado',
    '314': 'Bosque de galería y ripario',
    '315': 'Plantación forestal',
    '321': 'Herbazal',
    '322': 'Arbustal',
    '323': 'Vegetación secundaria',
    '331': 'Zonas arenosas naturales',
    '332': 'Afloramientos rocosos',
    '333': 'Tierras desnudas y degradadas',
    '334': 'Zonas quemadas',
    '335': 'Zonas glaciares y nivales',
    '411': 'Zonas pantanosas',
    '412': 'Turberas',
    '413': 'Vegetación acuática',
    '421': 'Pantanos costeros',
    '422': 'Salitral',
    '423': 'Sedimentos expuestos en bajamar',
    '511': 'Ríos (50 m)',
    '512': 'Lagunas, lagos y ciénagas',
    '513': 'Canales',
    '514': 'Cuerpos de agua artificiales',
    '521': 'Lagunas costeras',
    '522': 'Mares y océanos',
    '523': 'Estanques acuicultura marina'
}

# Asegurar que 'nivel_3' sea texto para mapear la leyenda
df['nivel_3'] = df['nivel_3'].astype(str)
df['leyenda'] = df['nivel_3'].map(leyendas)

# Filtrar solo clases válidas
df_filtrado = df[df['leyenda'].notnull()]

# Dividir en clases de alta y baja representación
tiles_1 = df_filtrado[df_filtrado['PCT_AREA'] >= 0.3]
tiles_2 = df_filtrado[df_filtrado['PCT_AREA'] < 0.3]

# Elegir tiles: 1 si alta representación, 2 si baja
sel_1 = (
    tiles_1.sort_values(['nivel_3', 'AREA_HECTAREAS'], ascending=[True, False])
    .groupby('nivel_3').head(1)
)

sel_2 = (
    tiles_2.sort_values(['nivel_3', 'AREA_HECTAREAS'], ascending=[True, False])
    .groupby('nivel_3').head(2)
)

# Unir resultados y quitar duplicados
seleccion_final = pd.concat([sel_1, sel_2]).drop_duplicates(subset=['tile_id', 'nivel_3'])

# Guardar como Excel
output_excel = r"C:\inteia\output\tiles_por_clase.xlsx"
seleccion_final.to_excel(output_excel, index=False)

print("✅ Excel generado con 1 o 2 tiles por clase, según representación.")


✅ Excel generado con 1 o 2 tiles por clase, según representación.


In [7]:
import arcpy
import pandas as pd
import os

# Leer Excel con los tile_id seleccionados
excel_path = r"C:\inteia\output\tiles_por_clase.xlsx"
df_tiles = pd.read_excel(excel_path)
selected_ids = df_tiles['tile_id'].astype(str).unique().tolist()

# Ruta del shapefile base
input_shp = r"C:\inteia\output\tiles_index.shp"

# Capa temporal para trabajar
arcpy.MakeFeatureLayer_management(input_shp, "tiles_layer")

# Armar cláusula SQL para seleccionar los tile_id deseados
id_field = "tile_id"
quoted_ids = [f"'{i}'" if not str(i).isdigit() else str(i) for i in selected_ids]
where_clause = f"{id_field} IN ({', '.join(quoted_ids)})"

# Aplicar selección
arcpy.management.SelectLayerByAttribute("tiles_layer", "NEW_SELECTION", where_clause)

# Exportar selección como nuevo shapefile
output_shp = r"C:\inteia\output\tiles_para_etiquetar.shp"
arcpy.management.CopyFeatures("tiles_layer", output_shp)

print("✅ Shapefile generado con tiles seleccionados en:", output_shp)


✅ Shapefile generado con tiles seleccionados en: C:\inteia\output\tiles_para_etiquetar.shp


# Subset

# sacar para dataset